In [ ]:
import geopandas as gpd
import requests
import json
import uuid
import os
from dotenv import load_dotenv, find_dotenv
from datetime import datetime

In [ ]:
load_dotenv(find_dotenv())
archive_path = os.getenv('ZIP_PATH')
file_path = os.getenv('SHP_PATH')
disk_token = os.getenv('YD_TOKEN')
local_file_path = os.getenv("LOCAL_DATA")
remote_file_path = os.getenv("REMOTE_DATA")
disk_base_folder = os.getenv("FOLDER_PATH")
disk_base_url = os.getenv("BASE_URL")

In [ ]:
def read_and_remove_archive(archive_path, file_path):
    try:
        gdf = gpd.read_file(f"zip://{archive_path}!{file_path}", encoding="utf-8")

        try:
            os.remove(archive_path)
            print("Shapefile обработан и удалён.")
        except FileNotFoundError:
            print("Shapefile для удаления не обнаружен.")

        return gdf

    except Exception as err:
        print(f"Ошибка при чтении Shapefile: {err}")
        return None

In [ ]:
gdf = read_and_remove_archive(archive_path, file_path)

if gdf is not None:
    print("Данные успешно загружены")
else:
    print("Ошибка при загрузке данных")

In [ ]:
def extract_geometry(geometry):
    geom_type = geometry.geom_type

    if geom_type == 'Polygon':
        return [
            [list(coord) for coord in geometry.exterior.coords],
            *[[list(coord) for coord in interior.coords] for interior in geometry.interiors]
        ]
    elif geom_type == 'MultiPolygon':
        return [
            [
                [list(coord) for coord in poly.exterior.coords],
                *[[list(coord) for coord in interior.coords] for interior in poly.interiors]
            ]
            for poly in geometry.geoms
        ]
    elif geom_type == 'LineString':
        return [list(coord) for coord in geometry.coords]
    elif geom_type == 'MultiLineString':
        return [list(list(coords) for coords in line.coords) for line in geometry.geoms]
    elif geom_type == 'Point':
        return list(geometry.coords[0])
    elif geom_type == 'MultiPoint':
        return [list(point.coords[0]) for point in geometry.geoms]
    elif geom_type == 'GeometryCollection':
        result = []
        for geom in geometry.geoms:
            try:
                result.append(extract_geometry(geom))
            except ValueError:
                continue
        return result
    else:
        raise ValueError(f"Неподдерживаемый тип геометрии: {geom_type}")

In [ ]:
output = {
    "paths": {},
    "points": {}
}

In [ ]:
for _, row in gdf.iterrows():
    try:
        geom = row.geometry
        geom_type = geom.geom_type

        if geom_type in ['Polygon', 'MultiPolygon']:
            coords = extract_geometry(geom)
            if isinstance(coords[0], list):
                for poly_data in coords:
                    poly_uuid = str(uuid.uuid4())
                    output["paths"][poly_uuid] = poly_data
            else:
                poly_uuid = str(uuid.uuid4())
                output["paths"][poly_uuid] = coords

        elif geom_type in ['LineString', 'MultiLineString', 'GeometryCollection']:
            coords = extract_geometry(geom)
            if isinstance(coords[0], list):
                for path_coords in coords:
                    poly_uuid = str(uuid.uuid4())
                    output["paths"][poly_uuid] = [path_coords]
            else:
                poly_uuid = str(uuid.uuid4())
                output["paths"][poly_uuid] = [coords]

        elif geom_type in ['Point', 'MultiPoint']:
            coords = extract_geometry(geom)
            if isinstance(coords[0], list):
                for point_coords in coords:
                    point_uuid = str(uuid.uuid4())
                    output["points"][point_uuid] = point_coords
            else:
                point_uuid = str(uuid.uuid4())
                output["points"][point_uuid] = coords

    except ValueError as e:
        print(f"Пропущена геометрия: {e}")
    except Exception as e:
        print(f"Ошибка при обработке строки: {e}")

In [ ]:
with open("index.json", "w", encoding="utf-8") as f:
    json.dump(output, f, ensure_ascii=False, indent=4)

In [ ]:
current_date_folder = datetime.now().strftime("%Y-%m-%d")
print(f"Папка для текущей даты: {current_date_folder}")

In [ ]:
check_and_create_root_folder(disk_base_folder, disk_token)

In [ ]:
check_or_create_data_folder(disk_base_folder, current_date_folder, disk_token)

In [ ]:
upload_to_ydisk(local_file_path, disk_base_folder, current_date_folder, disk_token)